# <span>Coletando os dados</span>
<hr style="border:2px solid #0077b9;">

<br/>

<div style="text-align: center;font-size: 90%;">
    Autor: Flávio Belizário da Silva Mota
    <br/><br/>
    Instituto Nacional de Pesquisas Espaciais (INPE)
    <br/>
    Avenida dos Astronautas, 1758, Jardim da Granja, São José dos Campos, SP 12227-010, Brazil
    <br/><br/>
    Contato: <a href="mailto:flavio.belizario.mota@gmail.com">flavio.belizario.mota@gmail.com</a>
    <br/>
    Professor: Rafael Santos
</div>

<br/>

<div style="text-align: justify;  margin-left: 25%; margin-right: 25%;">
<b>Objetivo.</b> Esse caderno Jupyter tem como objetivo apresentar a etapa de coleta dos dados para o projeto da disciplina CAP394 - Introdução à Ciência de Dados. Os dados são provenientes do projeto CRISIS NLP e foram coletados utilizando publicações do Twitter referentes à pandemia de COVID-19. O conjunto contém informações geolocalizadas a respeito da origem da publicação e  se no corpo da publicação existem referências a outros locais, também são apresentadas as informações sobre esses locais. Esses dados são disponibilizados em formato json e podem ser baixados em arquivos que representam as publicações diárias. Nessa etapa de coleta, foram consideradas publicações feitas no Brasil.
</div>

<br/>
<div style="text-align: center;font-size: 90%;">
    <b>Os dados podem ser encontrados em:</b>
    <div style="margin-left: 10px; margin-right: 10px">
    https://crisisnlp.qcri.org/covid19
    </div>
</div>

## O conjunto de dados
<hr style="border:2px solid #0077b9;">

<div>O conjunto de dados, chamado de GeoCoV19, foi elaborado através da coleta de dados do Twitter e contém mais de 524 milhões de <i>tweets</i> multilíngues coletados até 1º de maio de 2020. O conjunto contém cerca de 378 mil tweets com <i>geotags</i> e 5,4 milhões de <i>tweets</i> com informações sobre o local. Foram extraídos topônimos a partir do campo de localização do usuário e do conteúdo do <i>tweet</i> e essa informação foi transformada para geolocalizações, como país, estado ou cidade. Utilizando essa abordagem, 297 milhões de <i>tweets</i> foram associados a geolocalização usando o campo de localização do usuário e 452 milhões de <i>tweets</i> usando o conteúdo do <i>tweet</i>.</div>

<div>Os dados com informações geolocalizadas estão disponíveis para <i>download</i> em <a>https://crisisnlp.qcri.org/covid19</a> no formato <i>.zip</i>.

<figure>
    <img src='images/geo-download.png'/>
    <figcaption style='text-align: center;'>Figura 1 - Seção de download dos arquivos.</figcaption>
</figure> <br/>

Cada arquivo <i>.zip</i> contém um JSON com os <i>tweets</i> de um dia. Para coletar todas as publicações, é necessário baixar cada arquivo individualmente, por dia.</div>
    
<figure>
    <img src='images/geo-download-datas.png' style='max-width:400px; max-height:400px; width: auto; height: auto;'/>
    <figcaption style='text-align: center;'>Figura 2 - Seleção do arquivo com tweets do dia.</figcaption>
</figure> <br/>

## Automatizando a coleta dos dados
<hr style="border:2px solid #0077b9;">

<div>Realizar a coleta dessas publicações manualmente seria extremamente trabalhoso. Sendo assim, utilizaremos os códigos a seguir para automatizar essa coleta, construindo ao final um único arquivo JSON com as publicações do Brasil.</div>
<br/>
<div> <b>Mas de que forma faremos isso, se é preciso selecionar o arquivo que queremos baixar?</b> 
<br/> Inspecionando a página do projeto é possível descobrir que ao selecionar o botão de <i>download</i>, uma função é executada:
</div>
<br/>
<figure>
    <img src="images/insp-geo-download.png" style='max-width:700px; max-height:600px; width: auto; height: auto;'/>
    <figcaption style='text-align: center;'>Figura 3 - Inspecionando o botão de download.</figcaption>
</figure>

<br/>
<div>
    Descobrimos que a função <code>geo_pass_selected()</code> é chamada quando selecionamos o arquivo que queremos baixar. Inspecionando os arquivos fonte da página, conseguimos identificar o que essa função faz:
</div>
<br/>
<figure>
    <img src="images/insp-geo-pass.png" style='max-width:700px; max-height:600px; width: auto; height: auto;'/>
    <figcaption style='text-align: center;'>Figura 4 - Identificando a função <code>geo_pass_selected()</code>.</figcaption>
</figure>
<br/>
<div>
    Podemos notar que a função constrói um nome para o arquivo a ser baixado concatenando "geo_" e ".zip" a um valor que é obtido do elemento de seleção da página. Podemos inspecionar esse elemento para entender o que são esses valores:
</div>
<br/>
<figure>
    <img src="images/insp-geo-dates.png" style='max-width:900px; max-height:800px; width: auto; height: auto;'/>
    <figcaption style='text-align: center;'>Figura 5 - Inspecionando o componente de seleção do arquivo.</figcaption>
</figure>
<br/>
<div>
    Analisando o código javascript acima, podemos notar que os valores utilizados para preencher as opções do componente de seleção são gerados através de um laço de repetição que itera de uma data de início (01/02/2020) até uma data de fim (01/05/2020). Dessa forma, o nome do arquivo a ser baixado tem o formato "geo_2020-02-01.zip". A função em seguida concatena uma <i>url</i> <code>/covid_data/geo_files/</code> ao nome do arquivo para então disponibilizar o <i>download</i>. O arquivo baixado é um <i>.zip</i>, sendo necessário extrair seu conteúdo, para poder manipular o arquivo JSON contendo as publicações do dia.
</div>
<br/>
<div>
    Podemos utilizar essas informações para automatizar o processo de coleta dos dados, de acordo com os seguintes passos:
    <br/>
    <ul>
        <li>Criar uma lista de datas no intervalo 01/02/2020 até 01/05/2020;</li>
        <li>Iterar pelas datas dessa lista construindo o nome e <i>url</i> do arquivo a ser baixado;</li>
        <li>Realizar o <i>download</i> do arquivo;</li>
        <li>Extrair o conteúdo do arquivo <i>.zip</i> baixado;</li>
        <li>Analisar o arquivo JSON buscando pelas publicações feitas no Brasil;</li>
        <li>Gerar um novo arquivo JSON com essas publicações;</li>
        <li>Apagar o arquivo baixado para que ele não ocupe muito espaço em disco (considerando que os arquivos são grandes).</li>
    </ul>
</div>

### Instalando e importando as bibliotecas
<hr style="border:2px solid #0077b9;">


Instalando a biblioteca <code>wget</code> para realizar o download dos arquivos:

In [1]:
!pip install wget

Instalando a biblioteca <code>zipfile</code> para manipular os arquivos .zip:

In [2]:
!pip install zipfile36

Importando as bibliotecas necessárias:

In [23]:
import wget # biblioteca para download de arquivos
import json # biblioteca para manipulação de JSON
import pandas as pd # biblioteca para manipulação dos dados
import os # biblioteca para acessar recursos do sistema operacional
import zipfile # biblioteca para manipular arquivos .zip
from datetime import datetime # bibilioteca para manipular datas
import time # biblioteca para monitorar o tempo de execução

### Fluxo para carregamento dos dados
<hr style="border:2px solid #0077b9;">

Primeiro, vamos definir o intervalo de datas que será coletado. Para isso podemos empregar a bilblioteca <code>pandas</code>. As datas contemplam o intervalo de tempo entre 1º de Fevereiro de 2020 e 1º de Maio de 2020.

In [6]:
inicio = datetime.datetime.strptime("2020-02-01", "%Y-%m-%d")
fim = datetime.datetime.strptime("2020-05-01", "%Y-%m-%d")
datas_geradas = pd.date_range(inicio, fim) # gerando o intervalo
datas = datas_geradas.strftime("%Y-%m-%d").values # obtendo uma lista com as datas

Agora, iterando pela lista de datas, os arquivos correspondentes serão baixados empregando a biblioteca <code>wget</code>. Como estão em formato <i>.zip</i>, será necessário extrair o conteúdo desses arquivos. Utilizaremos a biblioteca <code>zipfile</code> para isso. Após a extração, é possível iterar pelas linhas do arquivo, obtendo o JSON correspondente de cada uma das linhas.
</br></br>
Na estrutura do JSON existem 8 chaves, como descrito na documentação dos dados, disponível <a href="https://crisisnlp.qcri.org/covid_data/geocov19_readme.txt">aqui</a>. As chaves são:
<ul>
    <li>tweet_id: o identificador único de um <i>tweet</i>;</li>
    <li>created_at: data e hora da criação do <i>tweet</i>, no formato UTC;</li>
    <li>user_id: identificador único do usuário que publicou o <i>tweet</i>;</li>
    <li>geo_source: este campo pode ter um dos quatro valores: (i) <code>coordinates</code>, (ii) <code>place</code>, (iii) <code>user_location</code> ou (iv) <code>tweet_text</code>. O valor depende da disponibilidade desses campos. No entanto, a prioridade é dada aos campos mais precisos, quando disponíveis. A ordem de prioridade é <code>coordinates</code>, <code>place</code>, <code>user_location</code> e <code>tweet_text</code>. Por exemplo, quando um <i>tweet</i> tem coordenadas de GPS, o valor será "coordinates", mesmo que todos os outros campos de localização estejam presentes. Se um <i>tweet</i> não tiver informações de GPS, place (local) e user_location (localização informada pelo usuário), o valor desse campo será "tweet_text" se houver alguma menção a localidades no texto do <i>tweet</i>.;</li>
</ul>

As demais chaves podem ser preenchidas com uma estrutura chamada <code>location_json</code>. Um exemplo dessa estrutura pode ser o seguinte:
<code>location_json: {"country_code":"us","state":"California","county":"San Francisco","city":"San Francisco"}</code>.
Dependendo da granularidade dos dados disponíveis, as chaves <code>country_code, state, county ou city</code> podem estar ausentes no <code>location_json</code>.

<ul>
    <li>user_location: pode conter um "location_json" conforme descrito acima ou um JSON vazio. Este campo usa os metadados da informação "local" do perfil de um usuário do Twitter e representa a localização declarada do usuário no formato de texto;</li>
    <li>geo: pode conter um "location_json" conforme descrito acima ou um JSON vazio. Representa o campo "geo" informado pelo Twitter;</li>
    <li>tweet_locations: este campo pode conter uma matriz de "location_json" conforme descrito acima (<code>[location_json1, location_json2]</code>) ou uma matriz vazia. Este campo usa o conteúdo do <i>tweet</i> (ou seja, a publicação real) para encontrar topônimos. Uma publicação pode ter várias menções a locais diferentes (ou seja, topônimos). É por isso que existe uma variedade de locais representando todos esses topônimos em um <i>tweet</i>. Por exemplo, em um <i>tweet</i> como "O Reino Unido tem mais de 65.000 mortes por #COVID19. Mais do que Qatar, Paquistão e Noruega.", há quatro menções de localização. A matriz <code>tweet_locations</code> deve representar esses quatro separadamente;</li>
    <li>place: este campo pode conter um "location_json" como descrito acima ou um JSON vazio. Ele representa o campo "lugar" fornecido pelo Twitter.</li>
</ul>

Considerando as informações acima, a estratégia adotada para coletar os dados foi encontrar em cada JSON aqueles que possuem o campo <code>geo_source</code> com os valores <code>coordinates</code>, <code>place</code> ou <code>user_location</code>, pois são os valores que indicam com maior precisão a localização de origem da publicação. Em seguida, consideramos o campo <code>user_location</code>. Caso esse campo não seja vazio, procuramos por uma chave <code>country_code</code> que contenha o valor br, uma vez que queremos publicações cujo país de origem seja o Brasil.
</br></br>
Por fim, adicionamos o JSON que atende às especificações ao conjunto de dados final e excluímos o arquivo que estava sendo analisado, para que um novo possa ser baixado, extraído e analisado, até que todas as datas da lista sejam obtidas. Vamos registrar o tempo necessário para executar todo esse processo.

In [26]:
print('Iniciando coleta em %s' % (datetime.now().strftime("%d/%m/%Y - %H:%M:%S")))
tac_geral = time.time()
for data in datas:
    json_final = []
    doc = "geo_"+ data +".zip"
    arq = "geo_"+ data +".json"
    doc_url = "https://crisisnlp.qcri.org/covid_data/geo_files/"+ doc
    print('Baixando arquivo: %s' % doc)
    tac = time.time()
    wget.download(doc_url)
    tic = time.time()
    print("Duração %.2f segundos" % (tic - tac))
    
    tac = time.time()
    with zipfile.ZipFile(doc, mode="r") as arquivo:
        with arquivo.open(arq, mode="r") as arq_json:
            for linha in arq_json:
                j_linha = json.loads(linha)
                if j_linha['geo_source'] == 'coordinates' or j_linha['geo_source'] == 'place' or j_linha['geo_source'] == 'user_location':
                    if len(j_linha['user_location']) != 0 and j_linha['user_location']['country_code'] == 'br':
                        json_final.append(j_linha)
            arq_json.close()
        arquivo.close()
    
    with open("geo_json_final.json", "w") as arquivo_json:
        json.dump(json_final, arquivo_json)
        arquivo_json.close()
    
    tic = time.time()    
    print("Duração da coleta dos dados do JSON %.2f segundos" % (tic - tac))
    
    if os.path.isfile(doc):
        os.remove(doc)
        print("Removendo arquivo %s" % doc)
    else: 
        print("Erro: %s - arquivo não encontrado" % doc)
    
    print('---------------------------------')
    break

tic_geral = time.time()
print("Duração total da coleta %.2f minutos" % ((tic_geral - tac_geral)/60))
print("Coleta finalizada em %s" % (datetime.now().strftime("%d/%m/%Y - %H:%M:%S")))

Iniciando coleta em 08/08/2022 - 20:39:57
Baixando arquivo: geo_2020-02-01.zip
Duração 29.6648 segundos
Duração da coleta dos dados do JSON 6.3799 segundos
Removendo arquivo geo_2020-02-01.zip
---------------------------------
Duração total da coleta 0.6008 minutos
Coleta finalizada em 08/08/2022 - 20:40:33
